<a href="https://colab.research.google.com/github/TheNewLearn/BigData_hw1/blob/main/bd_hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get -y install openjdk-8-jre-headless
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from google.colab import drive
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
spark = SparkSession.builder.master("local").getOrCreate()
sc = SparkContext.getOrCreate()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
openjdk-8-jre-headless is already the newest version (8u292-b10-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [29]:
from pyspark.sql.functions import max
drive.mount("/content/drive")
path = "/content/drive/My Drive/household_power_consumption.txt"
dataset = sc.textFile(path).map(lambda line: line.split(";"))
header = dataset.first()
dataset = dataset.filter(lambda row:row != header)
gap = dataset.map(lambda x: x[2]).filter(lambda x: x!="?")
maxgap = gap.reduce(lambda x,y: x if (float(x)>float(y)) else y)
mingap = gap.reduce(lambda x,y: x if (float(x)<float(y)) else y)
countgap = gap.count()


grp = dataset.map(lambda x: x[3]).filter(lambda x: x!="?")
maxgrp = grp.reduce(lambda x,y: x if (float(x)>float(y)) else y)
mingrp = grp.reduce(lambda x,y: x if (float(x)<float(y)) else y)
countgrp = grp.count()


vol = dataset.map(lambda x: x[4]).filter(lambda x: x!="?")
maxvol = vol.reduce(lambda x,y: x if (float(x)>float(y)) else y)
minvol = vol.reduce(lambda x,y: x if (float(x)<float(y)) else y)
countvol = vol.count()


gi = dataset.map(lambda x: x[5]).filter(lambda x: x!="?")
maxgi = gi.reduce(lambda x,y: x if (float(x)>float(y)) else y)
mingi = gi.reduce(lambda x,y: x if (float(x)<float(y)) else y)
countgi = gi.count()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
root
 |-- name: string (nullable = true)
 |-- count: long (nullable = true)
 |-- maximum: string (nullable = true)
 |-- minimum: string (nullable = true)

+---------------------+-------+-------+-------+
|name                 |count  |maximum|minimum|
+---------------------+-------+-------+-------+
|Global_active_power  |2049280|11.122 |0.076  |
|Global_reactive_power|2049280|1.390  |0.000  |
|Voltage              |2049280|254.150|223.200|
|Global_intensity     |2049280|48.400 |0.200  |
+---------------------+-------+-------+-------+



In [50]:
data = [("Global_active_power",countgap,maxgap,mingap),
        ("Global_reactive_power",countgrp,maxgrp, mingrp),
        ("Voltage",countvol,maxvol, minvol),
        ("Global_intensity",countgi,maxgi, mingi)]

columns = ["name","count","maximum","minimum"]

df = spark.createDataFrame(data=data,schema=columns)
df.printSchema()
df.show(truncate=False)

root
 |-- name: string (nullable = true)
 |-- count: long (nullable = true)
 |-- maximum: string (nullable = true)
 |-- minimum: string (nullable = true)

+---------------------+-------+-------+-------+
|name                 |count  |maximum|minimum|
+---------------------+-------+-------+-------+
|Global_active_power  |2049280|11.122 |0.076  |
|Global_reactive_power|2049280|1.390  |0.000  |
|Voltage              |2049280|254.150|223.200|
|Global_intensity     |2049280|48.400 |0.200  |
+---------------------+-------+-------+-------+



In [159]:
gapfloat = gap.map(lambda x:float(x))
meangap = gapfloat.mean()
stdgap = gapfloat.stdev()

grpfloat = grp.map(lambda x:float(x))
volfloat = vol.map(lambda x:float(x))
gifloat = gi.map(lambda x:float(x))

meangrp = grpfloat.mean()
stdgrp = grpfloat.stdev()

meanvol = volfloat.mean()
stdvol = volfloat.stdev()

meangi = gifloat.mean()
stdgi = gifloat.stdev()

In [49]:
print(stdgap,stdgap,stdvol,stdgi)
data2 = [("Global_active_power",meangap,str(stdgap)),
        ("Global_reactive_power",meangrp,str(stdgrp)),
        ("Voltage",meanvol,str(stdvol)),
        ("Global_intensity",meangi,str(stdgi))]

columns2 = ["name","mean","standard deviatio"]

df2 = spark.createDataFrame(data=data2,schema=columns2)
df2.printSchema()
df2.show(truncate=False)

1.05729390312673 1.05729390312673 3.2399858884915984 4.444395175406103
root
 |-- name: string (nullable = true)
 |-- mean: double (nullable = true)
 |-- standard deviatio: string (nullable = true)

+---------------------+-------------------+-------------------+
|name                 |mean               |standard deviatio  |
+---------------------+-------------------+-------------------+
|Global_active_power  |1.0916150365006068 |1.05729390312673   |
|Global_reactive_power|0.12371447630387154|0.11272195204788779|
|Voltage              |240.83985797450583 |3.2399858884915984 |
|Global_intensity     |4.627759310587101  |4.444395175406103  |
+---------------------+-------------------+-------------------+



In [158]:
from pyspark.sql.functions import col
from functools import reduce
from pyspark.sql import DataFrame
gapdf = gapfloat.map(lambda x: (x, )).toDF(["Global_active_power"])
gapdf2 = gapdf.withColumn("Min Max Normalization_Global_active_power",(col("Global_active_power")- float(mingap))/(float(maxgap)-float(mingap)))
gapdf2.show()

grpdf = grpfloat.map(lambda x: (x, )).toDF(["Global_reactive_power"])
grpdf2 = grpdf.withColumn("Min Max Normalization_Global_reactive_power",(col("Global_reactive_power")- float(mingrp))/(float(maxgrp)-float(mingrp)))
grpdf2.show()

voldf = volfloat.map(lambda x: (x, )).toDF(["Voltage"])
voldf2 = voldf.withColumn("Min Max Normalization_Voltage",(col("Voltage")- float(minvol))/(float(maxvol)-float(minvol)))
voldf2.show()

gidf = gifloat.map(lambda x: (x, )).toDF(["Global_intensity"])
gidf2 = gidf.withColumn("Min Max Normalization_Global_intensity",(col("Global_intensity")- float(mingi))/(float(maxgi)-float(mingi)))


gap_list = gapdf2.select("Min Max Normalization_Global_active_power").rdd.flatMap(lambda x: x).collect()
grp_list = grpdf2.select("Min Max Normalization_Global_reactive_power").rdd.flatMap(lambda x: x).collect()
vol_list = voldf2.select("Min Max Normalization_Voltage").rdd.flatMap(lambda x: x).collect()
gi_list = gidf2.select("Min Max Normalization_Global_intensity").rdd.flatMap(lambda x: x).collect()















+-------------------+-----------------------------------------+
|Global_active_power|Min Max Normalization_Global_active_power|
+-------------------+-----------------------------------------+
|              4.216|                       0.3747963063552418|
|               5.36|                       0.4783632084012313|
|              5.374|                       0.4796306355241717|
|              5.388|                      0.48089806264711216|
|              3.666|                       0.3250045265254391|
|               3.52|                        0.311787072243346|
|              3.702|                       0.3282636248415716|
|                3.7|                      0.32808256382400874|
|              3.668|                      0.32518558754300203|
|              3.662|                      0.32464240449031323|
|              4.448|                      0.39579938439254037|
|              5.412|                      0.48307079485786714|
|              5.224|                   

In [157]:
with open('/content/drive/My Drive/result.txt', 'w') as f:
  f.write("Min Max Normalization_Global_active_power;Min Max Normalization_Global_reactive_power;Min Max Normalization_Voltage;Min Max Normalization_Global_intensity")
  for i in range(0,len(gap_list)):
    f.write(str(gap_list[i])+";"+str(grp_list[i])+";"+str(vol_list[i])+ ";"+str(gi_list[i]))
    f.write('\n')
f.close()